In [1]:
import ast

import numpy as np
import pandas as pd

from sociodemographic_softlabels.run import map_annotators_to_groups
from sociodemographic_softlabels.datasets import kumar
from sociodemographic_softlabels.analysis.replicability import (
    significance_test_per_group,
    k_estimator_groups
)

SAMPLE_PATH = "../data/processed/kumar/sample_5000_annotators_v3.csv"
DATASET_PATH = 'kumar_dataset.json'

## Load Data

In [2]:
df = pd.concat([
    pd.read_csv('../experiments/0.2-baseline/result.csv'),
    pd.read_csv('../experiments/1.1-age/result.csv'),
    pd.read_csv('../experiments/1.2-gender/result.csv'),
    pd.read_csv('../experiments/1.3-education/result.csv'),
    pd.read_csv('../experiments/1.4-lgbq/result.csv'),
]).rename(lambda c: c if not c.startswith('eval_') else c[5:], axis = 1).reset_index(drop=True)

df['attributes'] = df['attributes'].map({
        "['ALL_0']": 'same layer',
        "['ALL_0', 'ALL_1', 'ALL_2', 'ALL_3', 'ALL_4']": '5 same layers',
        '[]': 'no extra layers',
        'Not applicable': 'single-task',
        "['gender']": 'gender',
        "['lgbtq_status']": 'lgbq',
        "['education']": 'education',
        "['age_range']": 'age',
        })

df[['predictions_per_annotator', 'labels_raw', 'labels_per_annotator',  'annotator_indecies']] = df[['predictions_per_annotator', 'labels_raw', 'labels_per_annotator',  'annotator_indecies']].applymap(lambda x: x.replace('nan', 'None')).applymap(ast.literal_eval).applymap(lambda x: np.array(x, dtype='float'))

In [3]:
all_data = kumar.Dataset.load(SAMPLE_PATH, metadata_path=DATASET_PATH)

to_groups = map_annotators_to_groups(
        all_data.annotators_mapping,
        all_data.sociodemographic_mapping,
        [
            "age_range",
            "lgbtq_status",
            "education",
            "gender"
        ]
    )

  0%|          | 0/23 [00:00<?, ?ba/s]

In [4]:
df[(df['random_seed'] == 2803636207) & (df['split'] == 0) & (df['attributes'] == 'no extra layers') & (df['do_random_assignment'] == False)]

,timestamp,loss,religion_important - Not too important - individual_0_precision,religion_important - Not too important - individual_0_recall,religion_important - Not too important - individual_0_f1-score,religion_important - Not too important - individual_0_support,religion_important - Not too important - individual_1_precision,religion_important - Not too important - individual_1_recall,religion_important - Not too important - individual_1_f1-score,religion_important - Not too important - individual_1_support,...,attributes,map_ids_to_attributes,setting,output_dir,while_train,do_stack_groups,do_random_assignment,group_layers_depth,random_seed,split
0,2023-05-18 14:17:53.858165,0.0,0.803314,0.808934,0.806114,1858,0.401349,0.392739,0.396997,606,...,no extra layers,[],K_FOLD,./models/custom_mt_model_multilabel,False,True,False,1,2803636207,0


## Significance Tests

### Gender

In [5]:
gender_p_dicts = significance_test_per_group(df, attribute_a='no extra layers', attribute_b='gender', to_groups=to_groups, attributes=['gender'])



Seed: 2803636207 - Split: 0 - gender - Group: Male
data shape:  (11861, 1)
sample size: 5930


bootstrap: 100%|████████████████████████████| 1000/1000 [00:15<00:00, 65.93it/s]



count sample diff f1   is twice tot diff f1....... 299  / 1000    p < 0.299  
count sample diff prec is twice tot diff prec..... 226  / 1000    p < 0.226  
count sample diff rec  is twice tot diff rec ..... 366  / 1000    p < 0.366  
count sample diff acc  is twice tot diff acc...... 196  / 1000    p < 0.196  


Seed: 2803636207 - Split: 0 - gender - Group: Female
data shape:  (13664, 1)
sample size: 6832


bootstrap: 100%|████████████████████████████| 1000/1000 [00:16<00:00, 60.26it/s]



count sample diff f1   is twice tot diff f1....... 713  / 1000    p < 0.713  
count sample diff prec is twice tot diff prec..... 818  / 1000    p < 0.818  
count sample diff rec  is twice tot diff rec ..... 543  / 1000    p < 0.543  
count sample diff acc  is twice tot diff acc...... 951  / 1000    p < 0.951  !


Seed: 2803636207 - Split: 0 - gender - Group: Nonbinary
data shape:  (120, 1)
sample size: 60


...


count sample diff f1   is twice tot diff f1....... 492  / 1000    p < 0.492  
count sample diff prec is twice tot diff prec..... 260  / 1000    p < 0.26   
count sample diff rec  is twice tot diff rec ..... 646  / 1000    p < 0.646  
count sample diff acc  is twice tot diff acc...... 175  / 1000    p < 0.175  


Seed: 2923262358 - Split: 3 - gender - Group: Female
data shape:  (13653, 1)
sample size: 6826


bootstrap: 100%|████████████████████████████| 1000/1000 [00:19<00:00, 51.58it/s]



count sample diff f1   is twice tot diff f1....... 931  / 1000    p < 0.931  
count sample diff prec is twice tot diff prec..... 863  / 1000    p < 0.863  
count sample diff rec  is twice tot diff rec ..... 969  / 1000    p < 0.969  !
count sample diff acc  is twice tot diff acc...... 565  / 1000    p < 0.565  


Seed: 2923262358 - Split: 3 - gender - Group: Nonbinary
data shape:  (120, 1)
sample size: 60


bootstrap: 100%|███████████████████████████| 1000/1000 [00:04<00:00, 217.68it/s]


count sample diff f1   is twice tot diff f1....... 809  / 1000    p < 0.809  
count sample diff prec is twice tot diff prec..... 804  / 1000    p < 0.804  
count sample diff rec  is twice tot diff rec ..... 722  / 1000    p < 0.722  
count sample diff acc  is twice tot diff acc...... 208  / 1000    p < 0.208  


In [6]:
gender_ks = k_estimator_groups(gender_p_dicts)
gender_ks

{'Nonbinary': {'k_count': 1, 'k_bonferroni': -1},
 'Male': {'k_count': -1, 'k_bonferroni': -1},
 'Female': {'k_count': 2, 'k_bonferroni': -1}}

In [8]:
gender_ks_df = pd.DataFrame(gender_ks).T.sort_index()
gender_ks_df = gender_ks_df.replace({-1: 0})
gender_ks_df = gender_ks_df.rename(columns={'k_count': '$\hat{k}_{count}$', 'k_bonferroni':'$\hat{k}_{Bonf.}$'})
gender_ks_df

,$\hat{k}_{count}$,$\hat{k}_{Bonf.}$
Female,2,0
Male,0,0
Nonbinary,1,0


In [9]:
gender_ks_df.to_latex('../tables/replication/baseline/gender.tex', escape=False)

#### Age

In [10]:
age_p_dicts = significance_test_per_group(df, attribute_a='no extra layers', attribute_b='age', to_groups=to_groups, attributes=['age_range'])



Seed: 2803636207 - Split: 0 - age_range - Group: 25 - 34
data shape:  (10313, 1)
sample size: 5156


bootstrap: 100%|████████████████████████████| 1000/1000 [00:14<00:00, 70.54it/s]



count sample diff f1   is twice tot diff f1....... 381  / 1000    p < 0.381  
count sample diff prec is twice tot diff prec..... 328  / 1000    p < 0.328  
count sample diff rec  is twice tot diff rec ..... 415  / 1000    p < 0.415  
count sample diff acc  is twice tot diff acc...... 301  / 1000    p < 0.301  


Seed: 2803636207 - Split: 0 - age_range - Group: 35 - 44
data shape:  (6220, 1)
sample size: 3110


bootstrap: 100%|████████████████████████████| 1000/1000 [00:10<00:00, 97.96it/s]



count sample diff f1   is twice tot diff f1....... 198  / 1000    p < 0.198  
count sample diff prec is twice tot diff prec..... 202  / 1000    p < 0.202  
count sample diff rec  is twice tot diff rec ..... 200  / 1000    p < 0.2    
count sample diff acc  is twice tot diff acc...... 203  / 1000    p < 0.203  


Seed: 2803636207 - Split: 0 - age_range - Group: 65 or older
data shape:  (675, 1)
sample size: 337


...


count sample diff f1   is twice tot diff f1....... 195  / 1000    p < 0.195  
count sample diff prec is twice tot diff prec..... 186  / 1000    p < 0.186  
count sample diff rec  is twice tot diff rec ..... 211  / 1000    p < 0.211  
count sample diff acc  is twice tot diff acc...... 156  / 1000    p < 0.156  


Seed: 2923262358 - Split: 3 - age_range - Group: 55 - 64
data shape:  (1903, 1)
sample size: 951


bootstrap: 100%|███████████████████████████| 1000/1000 [00:06<00:00, 144.42it/s]



count sample diff f1   is twice tot diff f1....... 922  / 1000    p < 0.922  
count sample diff prec is twice tot diff prec..... 894  / 1000    p < 0.894  
count sample diff rec  is twice tot diff rec ..... 941  / 1000    p < 0.941  
count sample diff acc  is twice tot diff acc...... 805  / 1000    p < 0.805  


Seed: 2923262358 - Split: 3 - age_range - Group: 45 - 54
data shape:  (3014, 1)
sample size: 1507


bootstrap: 100%|███████████████████████████| 1000/1000 [00:09<00:00, 109.31it/s]


count sample diff f1   is twice tot diff f1....... 472  / 1000    p < 0.472  
count sample diff prec is twice tot diff prec..... 325  / 1000    p < 0.325  
count sample diff rec  is twice tot diff rec ..... 567  / 1000    p < 0.567  
count sample diff acc  is twice tot diff acc...... 238  / 1000    p < 0.238  


In [11]:
age_ks = k_estimator_groups(age_p_dicts)
age_ks

{'65 or older': {'k_count': 1, 'k_bonferroni': -1},
 '25 - 34': {'k_count': 2, 'k_bonferroni': -1},
 '45 - 54': {'k_count': -1, 'k_bonferroni': -1},
 '35 - 44': {'k_count': 1, 'k_bonferroni': -1},
 '18 - 24': {'k_count': 2, 'k_bonferroni': -1},
 '55 - 64': {'k_count': 1, 'k_bonferroni': -1}}

In [12]:
age_ks_df = pd.DataFrame(age_ks).T.sort_index()
age_ks_df = age_ks_df.replace({-1: 0})
age_ks_df = age_ks_df.rename(columns={'k_count': '$\hat{k}_{count}$', 'k_bonferroni':'$\hat{k}_{Bonf.}$'})
age_ks_df

,$\hat{k}_{count}$,$\hat{k}_{Bonf.}$
18 - 24,2,0
25 - 34,2,0
35 - 44,1,0
45 - 54,0,0
55 - 64,1,0
65 or older,1,0


In [13]:
age_ks_df.to_latex('../tables/replication/baseline/age.tex', escape=False)

#### Education

In [14]:
education_p_dicts = significance_test_per_group(df, attribute_a='no extra layers', attribute_b='education', to_groups=to_groups, attributes=['education'])



Seed: 2803636207 - Split: 0 - education - Group: Bachelor's degree in college (4-year)
data shape:  (10439, 1)
sample size: 5219


bootstrap: 100%|████████████████████████████| 1000/1000 [00:16<00:00, 61.25it/s]



count sample diff f1   is twice tot diff f1....... 161  / 1000    p < 0.161  
count sample diff prec is twice tot diff prec..... 150  / 1000    p < 0.15   
count sample diff rec  is twice tot diff rec ..... 176  / 1000    p < 0.176  
count sample diff acc  is twice tot diff acc...... 140  / 1000    p < 0.14   


Seed: 2803636207 - Split: 0 - education - Group: High school graduate (high school diploma or equivalent including GED)
data shape:  (2130, 1)
sample size: 1065


bootstrap: 100%|███████████████████████████| 1000/1000 [00:06<00:00, 147.94it/s]



count sample diff f1   is twice tot diff f1....... 497  / 1000    p < 0.497  
count sample diff prec is twice tot diff prec..... 504  / 1000    p < 0.504  
count sample diff rec  is twice tot diff rec ..... 480  / 1000    p < 0.48   
count sample diff acc  is twice tot diff acc...... 493  / 1000    p < 0.493  


Seed: 2803636207 - Split: 0 - education - Group: Master's degree
data shape:  (3534, 1)
sample size: 1767


...


count sample diff f1   is twice tot diff f1....... 24   / 1000    p < 0.024  *
count sample diff prec is twice tot diff prec..... 32   / 1000    p < 0.032  *
count sample diff rec  is twice tot diff rec ..... 65   / 1000    p < 0.065  
count sample diff acc  is twice tot diff acc...... 10   / 1000    p < 0.01   *


Seed: 2923262358 - Split: 3 - education - Group: Less than high school degree
data shape:  (170, 1)
sample size: 85


bootstrap: 100%|███████████████████████████| 1000/1000 [00:04<00:00, 206.34it/s]



count sample diff f1   is twice tot diff f1....... 54   / 1000    p < 0.054  
count sample diff prec is twice tot diff prec..... 70   / 1000    p < 0.07   
count sample diff rec  is twice tot diff rec ..... 50   / 1000    p < 0.05   
count sample diff acc  is twice tot diff acc...... 97   / 1000    p < 0.097  


Seed: 2923262358 - Split: 3 - education - Group: Professional degree (JD, MD)
data shape:  (388, 1)
sample size: 194


bootstrap: 100%|███████████████████████████| 1000/1000 [00:04<00:00, 208.17it/s]


count sample diff f1   is twice tot diff f1....... 431  / 1000    p < 0.431  
count sample diff prec is twice tot diff prec..... 433  / 1000    p < 0.433  
count sample diff rec  is twice tot diff rec ..... 442  / 1000    p < 0.442  
count sample diff acc  is twice tot diff acc...... 442  / 1000    p < 0.442  


In [15]:
education_ks = k_estimator_groups(education_p_dicts)
education_ks

{'Doctoral degree': {'k_count': 2, 'k_bonferroni': -1},
 "Bachelor's degree in college (4-year)": {'k_count': 1, 'k_bonferroni': -1},
 'Professional degree (JD, MD)': {'k_count': -1, 'k_bonferroni': -1},
 'Less than high school degree': {'k_count': -1, 'k_bonferroni': -1},
 'High school graduate (high school diploma or equivalent including GED)': {'k_count': -1,
  'k_bonferroni': -1},
 'Some college but no degree': {'k_count': 2, 'k_bonferroni': 2},
 "Master's degree": {'k_count': -1, 'k_bonferroni': -1},
 'Associate degree in college (2-year)': {'k_count': -1, 'k_bonferroni': -1}}

In [16]:
education_ks_df = pd.DataFrame(education_ks).T.sort_index()
education_ks_df = education_ks_df.replace({-1: 0})
rename_mapping = {
        'Less than high school degree': 'Belowhigh school', 
        'Some college but no degree': 'College, no degree',
        'Associate degree in college (2-year)': 'Associate degree',
        "Bachelor's degree in college (4-year)":  "Bachelor's degree",
        'High school graduate (high school diploma or equivalent including GED)': 'High school',
        'Professional degree (JD, MD)': 'Professional degree'
    }
education_ks_df.index = education_ks_df.index.map(lambda i: rename_mapping[i] if i in rename_mapping else i)
education_ks_df = education_ks_df.rename(columns={'k_count': '$\hat{k}_{count}$', 'k_bonferroni':'$\hat{k}_{Bonf.}$'})
education_ks_df

,$\hat{k}_{count}$,$\hat{k}_{Bonf.}$
Associate degree,0,0
Bachelor's degree,1,0
Doctoral degree,2,0
High school,0,0
Belowhigh school,0,0
Master's degree,0,0
Professional degree,0,0
"College, no degree",2,2


In [17]:
education_ks_df.to_latex('../tables/replication/baseline/education.tex', escape=False)

### Sexual Orientation

In [18]:
lgbq_p_dicts = significance_test_per_group(df, attribute_a='no extra layers', attribute_b='lgbq', to_groups=to_groups, attributes=['lgbtq_status'])



Seed: 2803636207 - Split: 0 - lgbtq_status - Group: Heterosexual
data shape:  (22623, 1)
sample size: 11311


bootstrap: 100%|████████████████████████████| 1000/1000 [00:25<00:00, 39.79it/s]



count sample diff f1   is twice tot diff f1....... 1000 / 1000    p < 1.0    !
count sample diff prec is twice tot diff prec..... 1000 / 1000    p < 1.0    !
count sample diff rec  is twice tot diff rec ..... 1000 / 1000    p < 1.0    !
count sample diff acc  is twice tot diff acc...... 993  / 1000    p < 0.993  !


Seed: 2803636207 - Split: 0 - lgbtq_status - Group: Homosexual
data shape:  (739, 1)
sample size: 369


bootstrap: 100%|███████████████████████████| 1000/1000 [00:05<00:00, 190.58it/s]



count sample diff f1   is twice tot diff f1....... 669  / 1000    p < 0.669  
count sample diff prec is twice tot diff prec..... 675  / 1000    p < 0.675  
count sample diff rec  is twice tot diff rec ..... 653  / 1000    p < 0.653  
count sample diff acc  is twice tot diff acc...... 662  / 1000    p < 0.662  


Seed: 2803636207 - Split: 0 - lgbtq_status - Group: Bisexual
data shape:  (2418, 1)
sample size: 1209


...


count sample diff f1   is twice tot diff f1....... 16   / 1000    p < 0.016  *
count sample diff prec is twice tot diff prec..... 10   / 1000    p < 0.01   *
count sample diff rec  is twice tot diff rec ..... 37   / 1000    p < 0.037  *
count sample diff acc  is twice tot diff acc...... 1    / 1000    p < 0.001  **


Seed: 2923262358 - Split: 3 - lgbtq_status - Group: Homosexual
data shape:  (712, 1)
sample size: 356


bootstrap: 100%|███████████████████████████| 1000/1000 [00:05<00:00, 197.37it/s]



count sample diff f1   is twice tot diff f1....... 433  / 1000    p < 0.433  
count sample diff prec is twice tot diff prec..... 388  / 1000    p < 0.388  
count sample diff rec  is twice tot diff rec ..... 462  / 1000    p < 0.462  
count sample diff acc  is twice tot diff acc...... 382  / 1000    p < 0.382  


Seed: 2923262358 - Split: 3 - lgbtq_status - Group: Bisexual
data shape:  (2461, 1)
sample size: 1230


bootstrap: 100%|███████████████████████████| 1000/1000 [00:06<00:00, 152.00it/s]


count sample diff f1   is twice tot diff f1....... 46   / 1000    p < 0.046  *
count sample diff prec is twice tot diff prec..... 25   / 1000    p < 0.025  *
count sample diff rec  is twice tot diff rec ..... 53   / 1000    p < 0.053  
count sample diff acc  is twice tot diff acc...... 35   / 1000    p < 0.035  *


In [19]:
lgbq_ks = k_estimator_groups(lgbq_p_dicts)
lgbq_ks

{'Homosexual': {'k_count': 1, 'k_bonferroni': -1},
 'Bisexual': {'k_count': 2, 'k_bonferroni': -1},
 'Heterosexual': {'k_count': 4, 'k_bonferroni': 2}}

In [20]:
lgbq_ks_df = pd.DataFrame(lgbq_ks).T.sort_index()
lgbq_ks_df = lgbq_ks_df.replace({-1: 0})
lgbq_ks_df = lgbq_ks_df.rename(columns={'k_count': '$\hat{k}_{count}$', 'k_bonferroni':'$\hat{k}_{Bonf.}$'})
lgbq_ks_df

,$\hat{k}_{count}$,$\hat{k}_{Bonf.}$
Bisexual,2,0
Heterosexual,4,2
Homosexual,1,0


In [21]:
lgbq_ks_df.to_latex('../tables/replication/baseline/lgbq.tex', escape=False)